# Data Analysis for Eye Movements (PEECS) version 1.0b
### Copy this notebook to folder where your matlab main sequence data resides

### The following steps are required to analyze data:
1. Make sure prerequisites for analysis are taken care of (paths and installs) 
2. Verify files acquired from MonkeyLogic PEECS or psychopy and convert to MATLAB data files
3. Process matlab files:
    * go through files in the directory and save in memory which are OD, OS, or OU
    * concatenate the files if necessary
    * initial run through with Trial explorer (TE1)
4. Manual data exploration with Trial explorer (TE1)
5. Print the saccadograms

## Note: Monocular Vs. Binocular Recordings
Subjects have either been recorded monocularly or binocularly, pay attention to what files and datatypes they have!

For binocular recordings (moving forward) ML2 should follow the left or right eye which either covered with a patch or none at all. Signals are stored in Trials(n).SignalsOD or SignalsOS and saved in Signals(n,1:3), where n = 1 for OD and n=2 for OS signals.

For OLD Monocular recordings: We have been changing the ML2 control panel settings for "Eye" to either right eye or left eye signal from the Eyelink eye tracker. The "Eye Other" signal in ML2 is either sometimes blank or set to the opposite eye signal (which is always covered with a patch in monocular recordings thus not relevant). In these cases, when only 1 eye is recorded, we can ignore 'other eye' recording and set the monocular .Signal to the "Eye" signal in the bhv2 file. 


#### Prerequisites installations: MATLAB, Jupyter Notebook/Lab - Probably done yourself already
#### Prereq: add to search PATH in MATLAB: TE1/, TEGsharedlibs/, saccaddogram/

#### Version History:  
1.0b    - Updated the Saccadogram (version 7g) and this notebook to use helper files instead of code blocks.  
1.0a    - Most updated Saccadogram Notebook for Eyelink binocular recordings (8/2019)  
0.9.9.1 - Simplified Saccadogram so all files are in the subject root directory.  
0.9.9   - Binocular Data from MonkeyLogic Fix (LDS). Will run bhv2matML2 script. Other clean up of saccadogram.    
0.9.4 - 2/10/18 Edit. Works to produce inline and out of panel MATLAB figures.  
0.9.8   - bug fixes.   
0.9.7   - include extraction from raw bhv2 files (ML).  
0.9.6   - 2/2019  Update. For Monkey Logic.  
0.9.5   - 2/11/18 Edit. Updated ease of use for fixing problems in Trials structure during saccadogram printing & errors.  


___
## Step 0: (MonkeyLogic2/PEECS) Check PATH prerequisites
#### Also Look and display current Monkeylogic 2 (.bhv2) and MATLAB (.mat) files
If prerequistes are met, you will see "Found all 3 required paths! You can continue with analysis!"

Tips: To debug the path use command 'path' to view entire matlab path.

%  for the other eye
bhv2matML2

In [1]:
addpath ./s_help_scripts; % adds helper script path to matlab environment
% run script which checks for prerequisites
s0_checkprerequisites % this is a script in the helper folder which checks for the prerequisites for analysis


Output from cell above:
The Matlab PATH has the relevant folders below
IF TE1dev, TEGsharedlibs, or saccadogram are missing must add it to path in Matlab:

/Users/Linus/Dropbox/MATLAB/TEGsharedlibs
/Users/Linus/Dropbox/MATLAB/TE1dev2
/Users/Linus/Dropbox/MATLAB/0Patients/saccadogram

Found all 3 required paths! You can continue with analysis! 
 
Monkeylogic should already be installed, if not will get error of "file not found:"
Look for mlread and bhv2matML3 function scripts
/Users/Linus/Library/Mobile Documents/com~apple~CloudDocs/Documents/MATLAB/Add-Ons/Apps/NIMHMonkeyLogic2/mlread.m
/Users/Linus/Dropbox/MATLAB/0Patients/saccadogram/bhv2matML3.m

The current notebook fullpath is:  ans =
    '/Users/Linus/Dropbox/MATLAB/0Patients/Controls/191220_COPY'

BHV2 files found:
-rw-r--r-- 1 Linus staff 56M Dec 19 17:46 191219_CD_c_msq_as_sp_OU_JK_LDS_pl.bhv2

MATLAB files found:
gls: cannot access '*.mat': No such file or directory
No .mat files found!


___
### Step 1 (MonkeyLogic): Convert raw data file if needed, skip to Step 2 if already converted.

Assuming raw data files are stored in ../data/raw. Convert Monkey Logic Files to .mat files

#### Process the above files into .mat files. This is in the development but we are using some new scripts called "bhv2matML2" to convert these monkey logic raw data files to .mat files which should be stored here in the interim path directory:

This can be saved and used for future analysis Do this for OD and OS MSQ files:

- Using the "Select File to Open" Dialog, choose your raw data file.
- Next using the "Convert BHV2 file to Trials" dialog box, choose:
- Eye (OU/OD/OS), where OU is best for binocular monkey data. Deprecated: OD for files with only monocular right eye data stored in .eye and OS for files with OS data stored in .eye (rather than .eye2 as in binocular data)
- Half Screen (deprecated used for debugging only)
- debugging (for verbose output)
- Save File? 'y' if you want to save resulting Trials struct into .mat file of the same filename
- filename: confirm that the pathname and filename are the raw .bhv filename
- Save Director: confirm final folder if you do decide to 'y'es save the file into a .mat filename.

## Run this cell just to see which files you have and what you need to convert:

In [2]:
% If you run and re-run through notebook quickly with ctrl-Enter, you may accidentally overwrite previous data files, 
% this checks to see if previous .mat files have been created from .bhv2 files from ML2 for example
% ASSUMES FILES NAMES USE '_' underscore instead of spaces for delimiters
clear bhv2prefixes matfileprefixes
matfiles = dir('*.mat'); if isempty(matfiles); disp('No .mat files found!'); end;
bhv2files = dir('*.bhv2');

for j = 1:length(bhv2files)
    bhv2prefixes = split(bhv2files(j).name,' ');
    foundmatfileflag = 0;
    disp(['Looking for ' bhv2prefixes{1} ' in file names'])
    for i = 1:length(matfiles)
        matfileprefixes = split(matfiles(i).name,' ');
        if findstr(bhv2prefixes{1},matfileprefixes{1})
            fprintf(['Found matlab .mat file with prefix from .bhv2 file: ' matfiles(i).name '\n\n'])
            foundmatfileflag = 1;
        end
    end

    if foundmatfileflag == 0
        fprintf('\nFound a previous matlab file with prefix of .bhv2 ML2 file, so you can skip the next step (bhv2matML3)\n')
        fprintf('If you want to re-run bhv2matML3 must to it manually at the commandline.\n\n')
    else
        disp(['    Didn''t find corresponding MATLAB file, the next step is to run bhv2matML3 on this .bhv2 datafile: '])
        disp(['       ' bhv2files(j).name])
    end
end


No .mat files found!
Looking for 191219_CD_c_msq_as_sp_OU_JK_LDS_pl.bhv2 in file names

Found a previous matlab file with prefix of .bhv2 ML2 file, so you can skip the next step (bhv2matML3)
If you want to re-run bhv2matML3 must to it manually at the commandline.



In [3]:
% uncomment the following line and run this cell to convert .bhv2 file for the OD file:
bhv2matML4

Choose a data file
Running mlread from MonkeyLogic (please wait)
Filename: /Users/Linus/Dropbox/MATLAB/0Patients/Controls/191220_COPY/191219_CD_c_msq_as_sp_OU_JK_LDS_pl.bhv2
Confirmation of BHV2 eye data:
The following signals have valid eye movement data:
ans =
  1x4 cell array
    {'Eye X'}    {'Eye Y'}    {'Eye2 X'}    {'Eye2 Y'}
appears to be binocular recording
recEye is: OU
quick prompt
filepathnamesave =
    './191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat'
**Manual calibration is not necessary for monocular data**
Keeping the raw signal for the second eye
recEye =
    'OU'
inOU saving both eye positions signals
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,1

In [143]:
% uncomment the following line and run this cell to convert .bhv2 file for the OS file:
bhv2matML4

Choose a data file
Running mlread from MonkeyLogic (please wait)
Filename: /Users/Linus/Dropbox/MATLAB/0Patients/SaccadogramNew/190806_LB_OS_cond_msq_as_sp - JK_LDS2.bhv2
Confirmation of BHV2 eye data:
The following signals have valid eye movement data:
ans =
  1x2 cell array
    {'Eye X'}    {'Eye Y'}
recEye is: OS
quick prompt
filepathnamesave =
    './190806_LB_OS_cond_msq_as_sp - JK_LDS2_OS.mat'
**Manual calibration is not necessary for monocular data**
Keeping the raw signal for the second eye
recEye =
    'OS'
SAVING Trials to:
./190806_LB_OS_cond_msq_as_sp - JK_LDS2_OS.mat
Done


___
## Step 2:  ID MATLAB data files and label them OD, OS or OU in memory (saved to eyes{} cell array}

**Look to the files tab in Jupyter lab (left side tab) and verify your two or more .mat files are there, if working in jupyter/ipython notebook only see output below**

If you aren't using jupyter lab, but regular ipython/jupyter notebook verify the files below are in the same directory as this notebook:

In [4]:
% This script will help us identify which files are to be analyzed and label each file OD, OS, or OU depending on our experimental conditions
s2_labelOSODOU; 

*** .mat files in subject directory: ***
191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat  


***********************************************************************
*** LOOK FOR WINDOW IN THE BACK AND RAISE IT (Cmd-back appostrophe) ***
***********************************************************************
fnchose =
  1x1 cell array
    {'191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat'}
eyes =
  1x1 cell array
    {'OU'}
Now files and eyes have been labelled in memory, you can concatenate them in Step 3


## Step 3 Label Trials structure from files with .eye designation (OD, OS, OU) and CONCATENATE them if needed (ML2 or PEECS): 

### If you have more than one (monocular) data file, and you need to label each of those files as OD, OS, OU. If you have more than one: concatenate them with the following code (if you don't want to use this code, can use "catTrials" script on the command line - need to launch MATLAB desktop for this).

#### HELPFUL NOTES: 
- BEST WITH FRESH unevaluated by TE1 .mat files. 
- If you have already concantenated files, you should go straight to running TE1 (Step 5)
- You can use cattrials as well to concatenate files.
- With ML2 Trials, fixation break trials are included, there will be so many, so use the tool TrialsRmFB.m to delete those trials

In [8]:
s3_concatenate_files; % Script to label each Trial().eye with correct eye designation and then concatenate them into a single  Trials structure.

The Current Directory should be the notebooks directory if in a Jupyter lab, if not running a jupyter notebook, keep everything in same diretory
ans =
    '/Users/Linus/Dropbox/MATLAB/0Patients/Controls/191220_COPY'
***********************************************************************
*** LOOK FOR WINDOW IN THE BACK AND RAISE IT (Cmd-back appostrophe) ***
***********************************************************************
answer =
    'Yes'
eyes =
  1x1 cell array
    {'OU'}
dat = 
  struct with fields:

    Trials: [429x1 struct]
 
New Trials should appear here (if there is a vertcat error, may have to rerun bhv2matML script and specify OD and OS for eye):
  Name          Size                Bytes  Class     Attributes

  Trials      429x1             108249950  struct              



### Step 3.1 (MonkeyLogic2): many fixation breaks or non-reward trials can be erased, run the TrialsRmFB script on ML2 derived Trials struct in memory.

- Use Dialog box to run this script

In [14]:
% TO ELIMINATE BAD NON-REWARD TRIALS RUN THE TrialsRmFB.m tool
disp('*** LOOK FOR WINDOW TO RAISE ***')
TrialsRmFB;
% now save TrialsNew which have all the fixation breaks removed (1012 reward code only)
Trials = TrialsNew(:)

*** LOOK FOR WINDOW TO RAISE ***
Will keep trials in with ECODES:
        1012
Original Trials struct var name    :Trials
New created Trials struct var name : TrialsNew
 
...x.xxx..(10)..........(20)..........(30)..........(40).x........(50)...x......(60)
x.........(70)..........(80)..........(90)..........(100)..x.......(110)....xx...x(120)
..x.......(130)..x.x.x...(140)x.x..x..x.(150)...xxxxxxx(160)..........(170)x..xxxx..x(180)
..x.....x.(190)........x.(200).......x..(210)..xxxxxxxx(220)xxxxxxxxxx(230)xxxxxxxxxx(240)
xxxxxxxxxx(250)xxxxxxxxxx(260)xxxxxxxxxx(270)xxxxxxxxxx(280)xxxxxxxxxx(290)xxxxxxxxxx(300)
xxxxxxxxxx(310)xxxxxxxxxx(320)xxxxxxxxxx(330)xxxxxxxxxx(340)xxxxxxxxxx(350)xxxxxxxxxx(360)
xxxxxxxxxx(370)xxxxxxxxxx(380)xxxxxxxxxx(390)xxxxxxxxxx(400)xxxxxxxxxx(410)xxxxxxxxxx(420)
xxxxxxxxx  Name             Size                 Bytes  Class     Attributes

  Trials         429x1              108249950  struct              
  TrialsNew        1x176             36224528  struct  

### Step 3.2: Eliminate empty trials. Sometimes when ending the experiment early, an empty trial is created. These should be eliminated to not create errors in the main sequence.

In [16]:
elimlist = [];
for i = 1:length(Trials)
    if isempty(Trials(i).trialtype) 
        elimlist = [elimlist, i];
    end
end

if ~isempty(elimlist)
    disp('***********************************************************************')
    disp('*** LOOK FOR WINDOW IN THE BACK AND RAISE IT (Cmd-back appostrophe) ***')
    disp('***********************************************************************')

    elimlist
    dlgtxt = sprintf(['Would you like to eliminate these empty trials: ' num2str(elimlist) '?\n']);

    % QUICK PROMPT GUI to interact with Notebook rather than through cells:
    answer = questdlg(dlgtxt,'Empty Trials Elim','Yes','No','Yes')

    % Handle response
    switch answer
        case 'Yes'
            whos Trials
            Trials(elimlist)=[];
            whos Trials
            disp('Eliminated Empty Trials')
        case 'No'
            disp('Okay nothing done')
    end
else
    disp('No empty trials found, can continue to next step.')
end

No empty trials found, can continue to next step.


### Step 4. Save Trials and info into interm data file (name specified in Step 2)

In [17]:
fnchose_nomat = split(fnchose{1},'.');
savedfilename = [fnchose_nomat{1} '_OU_1012']; % This will change the file name after eliminating non reward trials in prev step.

disp('*** LOOK FOR WINDOW TO RAISE ***')
disp('*** LOOK FOR WINDOW TO RAISE ***')
disp('*** LOOK FOR WINDOW TO RAISE ***')

% QUICK PROMPT 
dims = [1 80];
GUItitle = 'Save Trials into File';
answer = inputdlg(['Do you want to save Trials into the following (change filename as needed) ?'],GUItitle,dims,{savedfilename});
% Handle response
if ~isempty(answer)    
    savedfilename = answer{1};
    save([savedfilename],'Trials');
    disp(['Saved Trials into  ' savedfilename ' in the local directory.']);
else
        disp('Cancelled');
        return;
end

pwd
!ls -Glph *.mat
sfn = dir([savedfilename '.mat']);
disp(' ')
disp(['The saved filename is: ''' sfn.name ''''])


*** LOOK FOR WINDOW TO RAISE ***
*** LOOK FOR WINDOW TO RAISE ***
*** LOOK FOR WINDOW TO RAISE ***
Saved Trials into  191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012 in the local directory.
ans =
    '/Users/Linus/Dropbox/MATLAB/0Patients/Controls/191220_COPY'
-rw-r--r-- 1 Linus 56M Dec 23 18:56 191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat
-rw-r--r-- 1 Linus 18M Dec 23 19:01 191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat
 
The saved filename is: '191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat'


### Step 5. Need to call some global variables in matlab BEFORE reload temp matlab file

In [18]:
%clear -except datafolder ;  % Clears all variables out, even global ones are emptied
global Trials i % This may inadvertently bring a filled out Trials variable to the global space.
whos Trials i
i = 1; Trials(1)

  Name          Size               Bytes  Class     Attributes

  Trials      176x1             36224528  struct    global    
  i             1x1                    8  double    global    

ans = 
  struct with fields:

      trialNumber: 1
      blockNumber: 1
    ML_Conditions: 1
        trialtype: 0
          outcome: 0
        SignalsOD: [1x3 struct]
        SignalsOS: [1x3 struct]
        Functions: [1x9 struct]
           Events: [1x11 struct]
           Frames: []
     InternalPars: NaN
            Units: []
       aStartTime: 0
       tStartTime: 0
         aEndTime: 2203
         tEndTime: 2203
          a2dRate: [1000 1000 1000 1000]
         datatype: 'MonkeyLogic'
          Signals: [2x3 struct]
              eye: 'OU'


### Step 6. (Get newfilename and datadir from Step 2 if needed), specify the matlab file which you wish to load which has the Trials structure and RELOAD IT (prevening global declaration to 'revert' to old variable Trials:

In [19]:
%
% YOU SHOULD PUT YOUR FILENAME HERE
%
%dir('*.mat')
%savedfilename
fullfilename = [savedfilename '.mat']
%fullfilename = ; % copy from output of step 4
%filename = 'Trialstemp_jupyter.mat';
dir(fullfilename)
disp('YOU SHOULD SEE A TRIALS STRUCTURE IN THE VARIABLE SPACE HERE:')
try
    %clear Trials
    load(fullfilename)
    whos Trials i
catch 
    disp('NO FILENAME FOUND: Did you include the .mat at the end of filename?')
end


fullfilename =
    '191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat'

191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat  


YOU SHOULD SEE A TRIALS STRUCTURE IN THE VARIABLE SPACE HERE:
  Name          Size               Bytes  Class     Attributes

  Trials      176x1             36224528  struct    global    
  i             1x1                    8  double    global    



# Trial Explorer (TE1) Instructions
## Step 7. use TE1dev to evaluate each and every trial
### At this point you should have openned a command window, it's easier to do this with matlab session launched from the command window from within Jupyter LAB. 
### use command 'runTE1dev' to launch it

### (deprecated) ONLY run this if it's the fisrt time before you went through structure with TrialExplorer1:

In [ ]:
%Trials_PEECS_Sgolay1000add % Running this script once will update the Trials structure to make TE1 run more quickly. By populating the sgolay filtered eye movement data

### TE1 Keyboard shortcuts
#### arrow keys = move forward and back
#### g = good trial
#### b = bad trial
#### q = quit, don't do this as you should leave it open and running as it will be useful when you print the saccadogram

### Instruction manual
### Note: the goal is to have 1 clean saccade after FP turns off (dotted black line). If a saccade occurs in the middle of the FP off, discard it as 'bad'. If no saccade is detected (or the saccade is missed) with a green-red vertical boundary, then choose manual sac1str and select beginning and end of that saccade exactly.
#### Run through each trial individually, if you see a lot of noise mark it 'bad'.
#### If you see a few randomly missing frames, especially during the saccade use '1 frame fill' button to fill those in, if the signal doesn't substantially improve then mark it as a 'bad' trial
#### To use the keyboard commands click in a 'blank' area of the GUI to un-highlight a button.
#### click on 'sgolay filtered data' to smooth data

### NOTE: This is best run from MATLAB desktop, right click in jupyter lab and choose New Console for Notebook, type "Desktop" in there.


In [20]:
%
% THIS Cell when run will start up the Trial explorer: it calls runTE1.m file which in turns calls some global variables and launches the Trial Explorer.
% Note that Trials and i are always global variables and all scripts here will have edit access to the Trials structure.
%
whos Trials i


disp('*************************************************************************')
disp('*** NOW IT''S TIME TO RUN Trial Explorer from the desktop: runTE1dev  ***')
disp('*************************************************************************')

%desktop % uncomment this will launch the matlabdesktop if not launched already
%runTE1dev % uncomment this will run TE1dev from Notebook, but it's better to run FROM THE DESKTOP (best choice)

disp('Instructions: Right click and select "New Console for Notebook" and execute "runTE1dev" in the new console window')
disp(' THEN click on bottom left button (Refreshall Toggle Slow)')

  Name          Size               Bytes  Class     Attributes

  Trials      176x1             36224528  struct    global    
  i             1x1                    8  double    global    

*************************************************************************
*** NOW IT'S TIME TO RUN Trial Explorer from the desktop: runTE1dev  ***
*************************************************************************
Instructions: Right click and select "New Console for Notebook" and execute "runTE1dev" in the new console window
 THEN click on bottom left button (Refreshall Toggle Slow)


### Once all trials have been evaluated save the trials structure for future analysis

** Mark down here the trial numbers which you have a question: **

e.g. I have a question with Trial number: 1,2,3 etc... Let me review this with the team later


In [22]:
TE1filename = [savedfilename 'TE1.mat']; % This will change the file name after eliminating non reward trials in prev step.

disp('*** LOOK FOR WINDOW TO RAISE ***')
disp('*** LOOK FOR WINDOW TO RAISE ***')
disp('*** LOOK FOR WINDOW TO RAISE ***')

% QUICK PROMPT 
dims = [1 80];
GUItitle = 'Save Trials into File';
answer = inputdlg(['Do you want to save Trials into the following (change filename as needed) ?'],GUItitle,dims,{TE1filename});
% Handle response
if ~isempty(answer)    
    TE1savedfilename = answer{1};
    save([TE1savedfilename],'Trials');
    disp(['Saved Trials into  ' savedfilename ' in the local directory.']);
else
        disp('Cancelled');
        return;
end

pwd
!ls -Glph *.mat
sfn = dir([TE1savedfilename]);
disp(' ')
disp(['The saved filename is: ''' sfn.name ''''])


*** LOOK FOR WINDOW TO RAISE ***
*** LOOK FOR WINDOW TO RAISE ***
*** LOOK FOR WINDOW TO RAISE ***
Saved Trials into  191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012 in the local directory.
ans =
    '/Users/Linus/Dropbox/MATLAB/0Patients/Controls/191220_COPY'
-rw-r--r-- 1 Linus 56M Dec 23 18:56 191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat
-rw-r--r-- 1 Linus 18M Dec 23 19:01 191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat
-rw-r--r-- 1 Linus 66M Dec 23 19:10 191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012TE1.mat
 
The saved filename is: '191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012TE1.mat'


# FINAL STEP: Saccadogram Data Plotting (really only meant for HUMAN Main Sequence Data)

At this point, with a fully analyzed Trials structure loaded in memory, you can run through it and produce a saccadogram. 

If no plots are generated and you end up seeing an error, then there is a problem with your Trials structure. The procedure to fix this and to clean up the saccadogram is to do the following: 

1. runTE1 to get the Trials Explorer to load and be able to edit Trials
2. Rerun saccadogram print on Trials in memory
3. On every error, refresh TE1 and fix the trial by manually seclecting the first saccade or marking the trial as [b]ad.
3. Once you're satisfied, save the Trials and info into a new filename.

Setting up variable names:

In [23]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% RUN THIS CELL if you do not have Trials structure loaded in memory (check with whos Trials i)      %
% Warning: If you haven't saved any changes to Trials in memory, running this cell will delete them! %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
clear global Trials i
global Trials i

%% THE LATEST PATIENT FILENAME HERE:
datafolder = './'
dir(datafolder)
ptfname = TE1savedfilename
load([datafolder ptfname])
whos Trials

datafolder =
    './'

.                                                       
..                                                      
.DS_Store                                               
.ipynb_checkpoints                                      
1.0b_Saccadogram_PEECS.ipynb                            
191219_CD_c_msq_as_sp_OU_JK_LDS_pl.bhv2                 
191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU.mat               
191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012.mat     
191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012TE1.mat  
README.md                                               
old                                                     
reports                                                 
s_help_scripts                                          


ptfname =
    '191219_CD_c_msq_as_sp_OU_JK_LDS_pl_OU_noSP_1012TE1.mat'
  Name          Size                Bytes  Class     Attributes

  Trials      176x1             100121150  struct    global    



In [29]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% THIS CELL PRINTS THE SACCADOGRAM                                        %
%                                                                         %
% IF YOU ENCOUNTER ERRORS IN PRINTING RE-RUN THIS CELL, be sure to have a %
% helpful Matlab Kernel console open as well: Right click and select      %
% "New Console for Notebook". Also to have a running instance of TE1 is   %                                            %
% necessary: "runTE1" shift-enter in new console to raise an instance.    %
% Follow the FINAL STEP directions above.                                 %   
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% Re-run this cell only if you run into errors and wish to eliminate bad trials or reanalyze them for
% the first saccade. The variable i, will be global and you can refresh TE1 to see the error generating trial.

% Setup variables
ttypes = [0]; % this chooses trial type 0 which is 10 degree saccade
ptexptdate = '12/20/19';
ptinitials = 'CD (DRAFT)';
patienttitle = ['Subject ' ptinitials ': 10 deg Saccadogram ',ptexptdate];
eyechoices = {'OD','OS'};
%eyechoices = {'OU'};
msaxislimit = [0 25 0 600];


savefileflag = 1; % if set to 1 will save pdfs to file
% Printing figures in separate matlab engine driven figures
% is better for debugging but if you want to print inline with this 
% notebook, use one of the following:
%imatlab_export_fig('')  % Native windows.
%imatlab_export_fig('print-png')  % Static png figures.

% The following is depreacted:
% if ptfname is empty, Eyemovclinical_Jupyter_v6 will use Trials struct in current memory!!!
% This is useful if errors are generated and you can use an active instance of TE1 GUI to fix that trial
% most often it is due to a problem with the first saccade made.
% Since i is global, click on 'refresh' in TE1 to evaluate that trial an fix it if possible.
% Then with ptfname set to '' (empty) re-run this cell to try to print out saccadogram
ptfname = 'TEST';
reportsfolder = './reports/'

figinfo.ptfname = ptfname;
figinfo.ptexptdate = ptexptdate;
figinfo.ptinitials = ptinitials;
figinfo.patienttitle1 = patienttitle;
figinfo.eyechoices = eyechoices;
figinfo.msaxlim = msaxislimit;
figinfo.darkmode = 0; % turn on dark mode for figures

save figinfo_lastprint.mat figinfo

% Run the saccadogram script:
% version 7updated for mOnkey Logic


reportsfolder =
    './reports/'


In [30]:
% Better run this in MATLAB IDE Command Window:
close all
Eyemovclinical_jupyter_v7g

NOTE: Trials struct needs to be run through REFRESH ALL at least once for a quick and dirty analysis
eyemovplots_clinicalv2f: eyechoice
plotselecteyetracev2g: R
idx:Filtering ttypes in idx
idx/emidx: Selecting/filtering for the correct eyemov directions
plotselecteyetrace2g: No eventstarttime given will use sac1start struct
1:1110 to 1710 max length Signal :2203TRS:
eye =
     1
 sac1_peakvel: 427.5288
8:874 to 1474 max length Signal :1940TRS:
eye =
     1
 sac1_peakvel: 376.0446
15:837 to 1437 max length Signal :1912TRS:
eye =
     1
 sac1_peakvel: 418.1129
17:822 to 1422 max length Signal :1890TRS:
eye =
     1
 sac1_peakvel: 395.8558
19:810 to 1410 max length Signal :1885TRS:
eye =
     1
 sac1_peakvel: 414.017
20:836 to 1436 max length Signal :1915TRS:
eye =
     1
 sac1_peakvel: 416.6831
21:707 to 1307 max length Signal :2033TRS:
eye =
     1
 sac1_peakvel: 294.6462
22:807 to 1407 max length Signal :1886TRS:
eye =
     1
 sac1_peakvel: 406.1441
23:814 to 1414 max length Signal :18

In [11]:
who


Your variables are:

contstartup   filesuffix    maxvelaxis    name          ptexptdate    sac1ovr_flag  ttypes        
eyechoices    harddrive     minvelaxis    patienttitle  ptinitials    savefileflag  



### Latency Analysis

In [3]:
for i = 1:length(Trials)
    Tc = [Trials(i).Events.Code];
    Tt = [Trials(i).Events.Time];
    FPofftime = Tt([Tc == 1005]);
    try
        latency = Trials(i).sac1str.start - FPofftime;
        Trials(i).latency = latency;
    catch
        Trials(i).errors.latency = 'No latency able to be calculated.';
    end
end
%only include latencies that are <500ms
lat = [Trials.latency];
lat = lat([lat<=500]);
figure
nhist(lat,'minbins',50,'median','linewidth',1)

> In nhist (line 716)
ans =
    'Plot #1: mean=202.05, std=74.11, median=194.81, '


## Some helpful functions and references below:


In [ ]:
help TEGsharedlibs

In [ ]:
help saccadogram

In [ ]:
!ls -Glpht ("/Users/Linus/Dropbox/MATLAB/\!Patients/")

In [ ]:
which eyemovplots_clinicalv2b